In [85]:
cd /home/hemanth/cnn-downscaling/src/

/home/hemanth/cnn-downscaling/src


In [86]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
import seaborn as sns

import torch
import torch.nn.functional as F

from training.data import WeatherBenchSuperresolutionDataModule
from training.models import LitSuperresolutionModelWrapper

In [88]:
plt.style.use('seaborn-pastel')
sns.set_style("ticks")

# CNN Saving the eval metrics Dataframe

In [133]:
best_ckpt = "/home/hemanth/cnn-downscaling/src/evaluation/cnn-pr-scaled.ckpt"

In [134]:
DATA_DIR = "/home/hemanth/downscale/generative-downscaling/data/processed/temp/"
COARSE_SUB_DIR = "5625/temp_5625_processed.zarr"  # 5.625 degrees
FINE_SUB_DIR = "1406/temp_1406_processed.zarr"  # 1.402 degrees

In [135]:
data_module = WeatherBenchSuperresolutionDataModule(
    coarse_dir = os.path.join(DATA_DIR, COARSE_SUB_DIR),
    fine_dir = os.path.join(DATA_DIR, FINE_SUB_DIR),
    batch_size = 1461  # length of test set
)

data_module.setup(stage=None)

test_dataloader = data_module.test_dataloader()
coarse_test, fine_test = iter(test_dataloader).next()

del test_dataloader

In [136]:
import zarr
model = LitSuperresolutionModelWrapper.load_from_checkpoint(best_ckpt)
model.eval()

LitSuperresolutionModelWrapper(
  (model): Sequential(
    (0): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1), padding=valid)
    (1): ReLU()
    (2): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1), padding=valid)
    (3): ReLU()
    (4): Conv2d(5, 13, kernel_size=(3, 3), stride=(1, 1), padding=valid)
    (5): ReLU()
    (6): Conv2d(13, 55, kernel_size=(3, 3), stride=(1, 1), padding=valid)
    (7): ReLU()
    (8): Conv2d(55, 21, kernel_size=(3, 3), stride=(1, 1), padding=valid)
    (9): ReLU()
    (10): Flatten(start_dim=1, end_dim=-1)
    (11): Linear(in_features=1344, out_features=398, bias=True)
    (12): Linear(in_features=398, out_features=4096, bias=True)
    (13): Unflatten(dim=1, unflattened_size=(64, 64))
  )
  (loss): MSELoss()
)

In [137]:
preds_test = model(coarse_test)
preds_test.shape

torch.Size([1461, 64, 64])

In [138]:
torch.set_grad_enabled(False);

In [139]:
train_dates, val_dates, test_dates = data_module.split_date_ranges

In [140]:
std = torch.from_numpy(data_module.daily_std['std'].values[-1461:]).view(1461,1,1)

In [141]:
mean = torch.from_numpy(data_module.daily_means['mean'].values[-1461:]).view(1461,1,1)

In [153]:
preds_test_unscaled = preds_test * std + mean
fine_test_unscaled = fine_test * std + mean
daily_mses = torch.mean((preds_test_unscaled - fine_test_unscaled)**2, dim=(1,2))
mse = F.mse_loss(preds_test_unscaled, fine_test_unscaled)
mse  # aligns with https://wandb.ai/6759-proj/cnn/runs/1uyqum0l

tensor(1.8556, dtype=torch.float64)

In [151]:
daily_mses.min(),daily_mses.mean(),daily_mses.max(),daily_mses.std()

(tensor(0.8333, dtype=torch.float64),
 tensor(1.8556, dtype=torch.float64),
 tensor(4.0361, dtype=torch.float64),
 tensor(0.5117, dtype=torch.float64))

In [154]:
#daily_mses = torch.mean((preds_test - fine_test)**2, dim=(1,2))
torch.mean(daily_mses)  # reproduces what's on W&B

tensor(1.8556, dtype=torch.float64)

In [155]:
pred_demeaned = (preds_test_unscaled - preds_test_unscaled.mean(axis=(1,2), keepdims=True))
true_demeaned = (fine_test_unscaled - fine_test_unscaled.mean(axis=(1,2), keepdims=True))

daily_corrs = \
    torch.sum(pred_demeaned*true_demeaned, axis=(1,2)) / \
    (torch.sqrt(torch.sum(pred_demeaned ** 2, axis=(1,2))) * torch.sqrt(torch.sum(true_demeaned ** 2, axis=(1,2))))

del pred_demeaned, true_demeaned

print(daily_corrs.min(), daily_corrs.mean(), daily_corrs.max())

tensor(0.9891, dtype=torch.float64) tensor(0.9960, dtype=torch.float64) tensor(0.9983, dtype=torch.float64)


In [156]:
train_dates, val_dates, test_dates = data_module.split_date_ranges

In [157]:
test_pdf = pd.DataFrame(
    fine_test.flatten(),
    index=data_module.fine.loc[test_dates[0]:test_dates[-1]].index,
    columns=["temp_true"]
)
test_pdf["temp_pred"] = preds_test.flatten()

In [158]:
test_pdf_unscaled = \
    test_pdf.join(data_module.daily_means).join(data_module.daily_std)

test_pdf_unscaled['temp_true'] = \
    test_pdf_unscaled['temp_true'] * test_pdf_unscaled['std'] + \
    test_pdf_unscaled['mean']

test_pdf_unscaled['temp_pred'] = \
    test_pdf_unscaled['temp_pred'] * test_pdf_unscaled['std'] + \
    test_pdf_unscaled['mean']

del test_pdf_unscaled['mean']
del test_pdf_unscaled['std']

In [159]:
def q02(ser):
    return ser.quantile(.02)

def q98(ser):
    return ser.quantile(.98)

test_biases = test_pdf_unscaled.groupby(
    [test_pdf_unscaled.index.get_level_values("date")]
).agg(['mean', q02, q98])

In [160]:
test_biases['bias_mean'] = test_biases['temp_true']['mean'] - test_biases['temp_pred']['mean']
test_biases['bias_q02'] = test_biases['temp_true']['q02'] - test_biases['temp_pred']['q02']
test_biases['bias_q98'] = test_biases['temp_true']['q98'] - test_biases['temp_pred']['q98']

In [161]:
test_tx = test_pdf_unscaled.groupby(
    [test_pdf.index.get_level_values("date").to_period('M'), "lat", "lon"]
).agg({
    "temp_true": [np.max, np.min],
    "temp_pred": [np.max, np.min],
})

test_tx.columns = ["true_txx", "true_txn", "pred_txx", "pred_txn"]

In [162]:
test_tx["bias_txx"] = test_tx["pred_txx"] - test_tx["true_txx"]
test_tx["bias_txn"] = test_tx["pred_txn"] - test_tx["true_txn"]

In [164]:
preds_std = torch.std(preds_test_unscaled, dim=(1,2))
fine_std = torch.std(fine_test_unscaled, dim=(1,2))

In [165]:
daily_stability = preds_std / fine_std

In [166]:
train_dates, val_dates, test_dates = data_module.split_date_ranges

In [167]:
eval_metrics = pd.DataFrame(
    {
        'mse' : daily_mses,
        'corr' : daily_corrs,
        'bias_p02' : test_biases['bias_q02'],
        'bias_mean' : test_biases['bias_mean'],
        'bias_p98' : test_biases['bias_q98'],
        'stability' : daily_stability,
    },
    index = test_dates
)

In [171]:
eval_metrics

,mse,corr,bias_p02,bias_mean,bias_p98,stability
date,,,,,,
2015-01-01,2.343767,0.997362,-0.226807,-0.075559,0.588721,0.999075
2015-01-02,2.439498,0.997352,0.005253,-0.056022,0.102429,0.997447
2015-01-03,2.491835,0.997423,-0.202788,-0.021836,0.654003,0.993647
2015-01-04,3.108826,0.996944,-0.723762,-0.000792,0.646341,0.987931
2015-01-05,2.731287,0.997154,-0.819950,-0.083551,0.027258,0.993719
...,...,...,...,...,...,...
2018-12-27,2.353531,0.997548,-1.010110,0.028425,0.103964,0.997591
2018-12-28,3.014554,0.996758,-0.925758,0.105319,0.640835,0.999908
2018-12-29,4.036123,0.995610,-1.482431,0.010554,0.878647,0.992313


In [172]:
eval_metrics.to_csv('eval_metrics_cnn.csv')

# Bilinear Interpolation

In [48]:
import zarr

In [49]:
bilinear_true = zarr.load('/home/hemanth/cnn-downscaling/src/evaluation/test_hr_true_bilinearInterpolation.zarr')
bilinear_pred = zarr.load('/home/hemanth/cnn-downscaling/src/evaluation/test_hr_pred_bilinearInterpolation.zarr')

In [54]:
bilinear_true = torch.from_numpy(bilinear_true['t2m'])
bilinear_pred = torch.from_numpy(bilinear_pred['t2m'])

In [56]:
daily_mses_bl = torch.mean((bilinear_pred - bilinear_true)**2,dim=(1,2))

In [57]:
daily_mses_bl.min(),daily_mses_bl.max(),daily_mses_bl.mean()

(tensor(3.6412, dtype=torch.float64),
 tensor(12.6106, dtype=torch.float64),
 tensor(6.4999, dtype=torch.float64))

In [58]:
pred_demeaned_bl = (bilinear_pred - bilinear_pred.mean(axis=(1,2),keepdims=True))
true_demeaned_bl = (bilinear_true - bilinear_true.mean(axis=(1,2),keepdims=True))

daily_corrs_bl =  torch.sum(pred_demeaned_bl*true_demeaned_bl, axis=(1,2)) /(torch.sqrt(torch.sum(pred_demeaned_bl ** 2, axis=(1,2))) * torch.sqrt(torch.sum(true_demeaned_bl ** 2, axis=(1,2))))

del pred_demeaned_bl, true_demeaned_bl

print(daily_corrs_bl.min(), daily_corrs_bl.mean(), daily_corrs_bl.max())

tensor(0.9328, dtype=torch.float64) tensor(0.9836, dtype=torch.float64) tensor(0.9949, dtype=torch.float64)


In [60]:
preds_p02_bl = torch.quantile(bilinear_pred.reshape(bilinear_pred.shape[0], -1), dim=1, q=.02)
preds_p98_bl = torch.quantile(bilinear_pred.reshape(bilinear_pred.shape[0], -1), dim=1, q=.98)

fine_p02_bl = torch.quantile(bilinear_true.reshape(bilinear_pred.shape[0], -1), dim=1, q=.02)
fine_p98_bl = torch.quantile(bilinear_true.reshape(bilinear_pred.shape[0], -1), dim=1, q=.98)

In [61]:
daily_biases_mean_bl =  bilinear_pred.mean(axis=(1,2)) - bilinear_true.mean(axis=(1,2))
daily_biases_p02_bl = preds_p02_bl - fine_p02_bl
daily_biases_p98_bl = preds_p98_bl - fine_p98_bl

In [62]:
torch.mean(daily_mses_bl)

tensor(6.4999, dtype=torch.float64)

In [63]:
preds_std_bl = torch.std(bilinear_pred, dim=(1,2))
fine_std_bl = torch.std(bilinear_true, dim=(1,2))

In [64]:
daily_stability_bl = preds_std_bl/fine_std_bl

In [65]:
daily_stability_bl

tensor([0.9932, 0.9950, 0.9926,  ..., 0.9844, 0.9849, 0.9915],
       dtype=torch.float64)

In [82]:
eval_metrics_bl = pd.DataFrame(
    {
        'mse' : daily_mses_bl,
        'corr' : daily_corrs_bl,
        'bias_p02' : daily_biases_p02_bl,
        'bias_mean' : daily_biases_mean_bl,
        'bias_p98' : daily_biases_p98_bl,
        'stability' : daily_stability_bl,
    },
    index = test_dates
)

In [83]:
eval_metrics_bl.to_csv('eval_metrics_bl.csv')